In [931]:
# Data Quality Analysis and Cleaning Script
# This script analyzes the MERU Data Insights Technical Assessment datasets,
# identifies data quality issues, and applies appropriate cleaning steps

In [854]:
import pandas as pd
import numpy as np
from datetime import datetime
import re


In [ ]:
#Load Historical Sales Data 

In [856]:

try:
    historical_sales = load_and_inspect_dataset("Assignment-Historical Sales.xlsx", sheet_name="Data")
    
    # Sample the data
    print("\nSample of Historical Sales data:")
    print(historical_sales.head(3))
    
    # Check data types
    print("\nData types of Historical Sales columns:")
    print(historical_sales.dtypes)
    
except Exception as e:
    print(f"ERROR loading Historical Sales data: {e}")
    historical_sales = pd.DataFrame()

# ----- Load Case Weights Data -----
try:
    case_weights = load_and_inspect_dataset("Assignment-CaseWeights.xlsx", sheet_name="Net Wt")
    
    # Sample the data
    print("\nSample of Case Weights data:")
    print(case_weights.head(3))
    
    # Check data types
    print("\nData types of Case Weights columns:")
    print(case_weights.dtypes)
    
except Exception as e:
    print(f"ERROR loading Case Weights data: {e}")
    case_weights = pd.DataFrame()

# ----- Load Budget Data -----
# The budget file has a non-standard header structure, so we'll first examine it without headers
try:
    budget_raw = pd.read_excel("Assignment-Budget.xlsx", sheet_name="Data", header=None)
    
    # Print the first few rows to identify the header row
    print("\nFirst 5 rows of Budget data (without headers):")
    print(budget_raw.head(5))
    
    # Based on visual inspection, row 4 appears to be the header row
    header_row = 4
    budget = load_and_inspect_dataset("AssignmentBudget.xlsx", sheet_name="Data", header_row=header_row)
    
    # Sample the data
    print("\nSample of Budget data after setting header row:")
    print(budget.head(3))
    
except Exception as e:
    print(f"ERROR loading Budget data: {e}")
    budget_raw = pd.DataFrame()
    budget = pd.DataFrame()




Loading Assignment-Historical Sales.xlsx...
File loaded successfully: Assignment-Historical Sales.xlsx
Dimensions: 17133 rows x 14 columns
Columns: Unnamed: 0, Unnamed: 1, Unnamed: 2, Unnamed: 3, Unnamed: 4, Unnamed: 5, Unnamed: 6, Unnamed: 7, Unnamed: 8, Unnamed: 9, Unnamed: 10, Unnamed: 11, Unnamed: 12, Unnamed: 13

Columns with missing values:
  - Unnamed: 10: 4 missing values (0.02%)
  - Unnamed: 11: 61 missing values (0.36%)

Found 1 duplicate rows (0.01%)

Sample of Historical Sales data:
  Unnamed: 0 Unnamed: 1    Unnamed: 2     Unnamed: 3 Unnamed: 4  \
0      Month       Year      Combined  Business Line   ItemCode   
1          1       2020  700002:15040     Business A      15040   
2          1       2020  700007:15040     Business A      15040   

      Unnamed: 5 Unnamed: 6 Unnamed: 7    Unnamed: 8  Unnamed: 9 Unnamed: 10  \
0  Customer code   Quantity  Total Amt       Channel  Item Group  Branded/PL   
1         700002       3840    41241.6  Food Service    Crackers     B

In [857]:
# Standardize column names: Remove extra spaces and drop any extraneous header rows if necessary.
historical_sales.columns = historical_sales.columns.str.strip()
# If the very first row contains column names, ensure to drop it if it’s duplicated:
historical_sales = historical_sales.iloc[1:].reset_index(drop=True)
historical_sales = historical_sales.drop_duplicates()

In [933]:
#historical_sales.head()

In [932]:
#historical_sales.shape

In [860]:
# Rename columns to standard names
historical_sales.columns = [
    "Month", "Year", "CombinedCode", "Business", "ProductCode", 
    "CustomerCode", "Boxes", "Revenue", "Channel", "ProductGroup", 
    "ProductCategory", "State", "WeightPerBox", "TotalWeight"
]

In [861]:
#Dropping combined code and re ordering the columns
# Drop the "CombinedCode" column from the sales table if it exists
if "CombinedCode" in historical_sales.columns:
    historical_sales.drop(columns=["CombinedCode"], inplace=True)

# Verify the column is removed
print("Remaining columns in Fact_Sales:", historical_sales.columns)


Remaining columns in Fact_Sales: Index(['Month', 'Year', 'Business', 'ProductCode', 'CustomerCode', 'Boxes',
       'Revenue', 'Channel', 'ProductGroup', 'ProductCategory', 'State',
       'WeightPerBox', 'TotalWeight'],
      dtype='object')


In [862]:
# Define the new column order
new_column_order = [
    "Year", "Month", "Business", "Channel","CustomerCode","State","ProductCode",
    "ProductCategory", "ProductGroup","Boxes","WeightPerBox","TotalWeight", "Revenue"]

# Reorder the columns
historical_sales = historical_sales[new_column_order]

# Verify the new order
print(historical_sales.head())


   Year Month    Business       Channel CustomerCode State ProductCode  \
0  2020     1  Business A  Food Service       700002    MA       15040   
1  2020     1  Business A  Food Service       700007    MI       15040   
2  2020     1  Business A  Food Service       700052    NY       15040   
3  2020     1  Business A  Food Service       700062    NH       10041   
4  2020     1  Business A  Food Service       700062    NH       15040   

  ProductCategory ProductGroup Boxes WeightPerBox TotalWeight    Revenue  
0         Branded     Crackers  3840         4.69     18009.6    41241.6  
1         Branded     Crackers  4800         4.69     22512.0      56016  
2         Branded     Crackers  3072         4.69    14407.68   35850.24  
3         Branded     Crackers    80           10         800       1632  
4         Branded     Crackers  9792         4.69    45924.48  124162.56  


In [863]:
missing_before = historical_sales.isna().sum()
print(missing_before)

Year                0
Month               0
Business            0
Channel             0
CustomerCode        0
State              61
ProductCode         0
ProductCategory     4
ProductGroup        0
Boxes               0
WeightPerBox        0
TotalWeight         0
Revenue             0
dtype: int64


In [864]:
print(historical_sales.dtypes)

Year               object
Month              object
Business           object
Channel            object
CustomerCode       object
State              object
ProductCode        object
ProductCategory    object
ProductGroup       object
Boxes              object
WeightPerBox       object
TotalWeight        object
Revenue            object
dtype: object


In [865]:
# Define optimal data types for each column
data_type_mapping = {
    "CustomerCode": "category",  # Categorical data (reduces memory usage)
    "ProductCode": "category",      # Categorical data
    "Business": "category",      # Business line
    "ProductGroup": "category",     # Product grouping
    "Boxes": "int32",         # Integer values for units sold
    "Revenue": "float32",       # Monetary values as float         
    "Channel": "category",       # Sales channel (Retail, Foodservice, etc.)
    "ProductCategory": "category", # Branded vs. Private Label
    "TotalWeight": "float32",    # Float for calculated total weight
    "WeightPerBox": "float32",  # Product weight per case
    "Month": "int8",             # Small integer (1-12)
    "Year": "int16",             # Small integer for years (1900-2100)
    "TransactionType": "category" # Transaction type classification
}

# Apply data type conversions
try:
    
    # Handle zeros before conversion - important to avoid the 1000+ issue
    for col in ['Boxes', 'Revenue']:
        # Keep track of zero values
        zero_mask = historical_sales[col] == 0
        
    # Apply optimized data types
    for column, dtype in data_type_mapping.items():
        if column in historical_sales.columns:
            # For numeric columns, handle NaN values first
            if dtype in ['int8', 'int16', 'int32', 'int64']:
                # Convert to float first, then to integer to handle NaN values
                historical_sales[column] = historical_sales[column].fillna(0).astype(dtype)
            else:
                historical_sales[column] = historical_sales[column].astype(dtype)
    
    # Restore zeros that might have been affected
    for col in ['Boxes', 'Revenue']:
        historical_sales.loc[zero_mask, col] = 0
    
    # Verify the conversion worked correctly
    print("\nData types after conversion:")
    print(historical_sales.dtypes)
    
    # Verify zero counts are preserved
    for col in ['Boxes', 'Revenue']:
        zero_count = (historical_sales[col] == 0).sum()
        print(f"Zero count in {col}: {zero_count}")

except Exception as e:
    print(f"Error during data type conversion: {e}")


Data types after conversion:
Year                  int16
Month                  int8
Business           category
Channel            category
CustomerCode       category
State                object
ProductCode        category
ProductCategory    category
ProductGroup       category
Boxes                 int32
WeightPerBox        float32
TotalWeight         float32
Revenue             float32
dtype: object
Zero count in Boxes: 2919
Zero count in Revenue: 2899


/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_35897/825477433.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  historical_sales[column] = historical_sales[column].fillna(0).astype(dtype)
/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_35897/825477433.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  historical_sales[column] = historical_sales[column].fillna(0).astype(dtype)


In [866]:
# Verify updated data types
print(historical_sales.dtypes)

Year                  int16
Month                  int8
Business           category
Channel            category
CustomerCode       category
State                object
ProductCode        category
ProductCategory    category
ProductGroup       category
Boxes                 int32
WeightPerBox        float32
TotalWeight         float32
Revenue             float32
dtype: object


In [867]:
#So in terms of missing values - We have ProductCategory and State; 

In [868]:
#Filling missing state values

In [869]:
# Step 1: Identify missing State values
missing_state_mask = historical_sales["State"].isnull()

# Step 2: Fill missing State with the most common State for the same CustomerCode
historical_sales["State"] = historical_sales.groupby("CustomerCode")["State"].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
)

# Step 3: If a State is still missing, fill with "Unknown"
historical_sales["State"].fillna("Unknown", inplace=True)

# Verify if all missing values are handled
print(f"Remaining missing State values: {historical_sales['State'].isnull().sum()}")


Remaining missing State values: 0


/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_35897/3651363465.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  historical_sales["State"] = historical_sales.groupby("CustomerCode")["State"].transform(
/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_35897/3651363465.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object

In [934]:
# Data type conversions

In [871]:
print(historical_sales[historical_sales["State"].isnull()])
#Interesting to note that if I removed all these rows, I;ll lose the rows where there has been a sale. 


Empty DataFrame
Columns: [Year, Month, Business, Channel, CustomerCode, State, ProductCode, ProductCategory, ProductGroup, Boxes, WeightPerBox, TotalWeight, Revenue]
Index: []


In [872]:

# Identify problematic records:
# Negative Quantity
neg_qty = historical_sales[historical_sales["Boxes"] < 0]
# Zero Quantity with Nonzero TotalAmt
zero_qty_nonzero_amt = historical_sales[(historical_sales["Boxes"] == 0) & (historical_sales["Revenue"] > 0)]
# Zero Quantity with Zero TotalAmt (canceled orders, test entries)
zero_qty_zero_amt = historical_sales[(historical_sales["Boxes"] == 0) & (historical_sales["Revenue"] == 0)]

print("Negative Qty records:", len(neg_qty))
print("Zero Qty with nonzero Amt records:", len(zero_qty_nonzero_amt))
print("Zero Qty & Zero Amt records:", len(zero_qty_zero_amt))

Negative Qty records: 6
Zero Qty with nonzero Amt records: 20
Zero Qty & Zero Amt records: 2899


In [873]:
# Remove rows where Quantity is negative
historical_sales = historical_sales[historical_sales["Boxes"] >= 0]

# Verify if negative quantities were removed
print("Negative Quantity Count:", (historical_sales["Boxes"] < 0).sum())


Negative Quantity Count: 0


In [874]:
# Flag cases where Quantity = 0 and TotalAmt > 0
historical_sales["TransactionType"] = historical_sales.apply(
    lambda row: "Service Charge/Adjustment" if (row["Boxes"] == 0 and row["Revenue"] > 0) else "Normal Sale", axis=1
)

# Verify the flagged cases
flagged_cases = historical_sales[historical_sales["TransactionType"] == "Service Charge/Adjustment"]



In [876]:
print(flagged_cases)
# Count the number of flagged transactions
flagged_count = (historical_sales["TransactionType"] == "Service Charge/Adjustment").sum()
print(f"Number of flagged transactions: {flagged_count}")



       Year  Month    Business       Channel CustomerCode State ProductCode  \
10472  2018      1  Business B  Food Service       760261    MI       97101   
10476  2018      1  Business B  Food Service       760261    MI       97102   
10480  2018      1  Business B  Food Service       760261    MI       97103   
10483  2018      1  Business B  Food Service       760261    MI       97128   
10492  2018      1  Business B  Food Service       760261    MI       97501   
13310  2018      6  Business B        Retail       760215    IA       97254   
13311  2018      6  Business B        Retail       760215    IA       97501   
13312  2018      6  Business B        Retail       760215    IA       97519   
13313  2018      6  Business B        Retail       760215    IA       97524   
13314  2018      6  Business B        Retail       760215    IA       97526   
13568  2018      6  Business B  Food Service       793067    MN       97100   
13571  2018      6  Business B  Food Service       7

In [877]:
#I am also removing cases where qty and amount are both 0 - mostly invalid transactions

In [878]:
# Step 6: Remove Zero Quantity & Zero Amount Transactions (Likely Invalid Records)
historical_sales = historical_sales[~((historical_sales["Boxes"] == 0) & (historical_sales["Revenue"] == 0))]


In [879]:
#I have so far handled all the data inconsisencies.

In [880]:
print("Total records:", len(historical_sales))

Total records: 14226


In [885]:
#historical_sales.head()

In [886]:
# Standardizing Channel Values
channel_mapping = {
    "Distributor": "Food Service",
    "Retail Private Label": "Retail"
}

historical_sales["Channel"] = historical_sales["Channel"].replace(channel_mapping)


/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_35897/3250674973.py:7: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  historical_sales["Channel"] = historical_sales["Channel"].replace(channel_mapping)


In [887]:
print(historical_sales["Channel"].unique())


['Food Service', 'Retail']
Categories (2, object): ['Food Service', 'Retail']


In [888]:
historical_sales.shape

(14226, 14)

In [889]:
historical_sales.to_csv("historical_sales_final.csv", index=False)


In [890]:
#There are cases where weight per case is 0
#There are cases where qty is now 0 but some sale amount is there - this includes the special cases and just one customer id 

In [891]:
#Moving on to handling case weights


In [892]:
case_weights = pd.read_excel("Assignment-CaseWeights.xlsx", sheet_name="Net Wt")

In [893]:
case_weights.shape

(397, 2)

In [894]:
case_weights.describe

<bound method NDFrame.describe of     Unnamed: 0        Unnamed: 1
0          NaN               NaN
1     Item No.  Case Weight (lb)
2          410                 2
3          440               2.5
4          450               2.3
..         ...               ...
392      97968               9.9
393      97969               2.8
394      97970               2.3
395      97971               2.3
396      97972               2.3

[397 rows x 2 columns]>

In [895]:
# Standardize column names: Remove extra spaces and drop any extraneous header rows if necessary.
case_weights.columns = case_weights.columns.str.strip()
# If the very first row contains column names, ensure to drop it if it’s duplicated:
case_weights = case_weights.iloc[1:].reset_index(drop=True)

case_weights = case_weights.drop_duplicates()


In [896]:
case_weights.head()

,Unnamed: 0,Unnamed: 1
0,Item No.,Case Weight (lb)
1,410,2
2,440,2.5
3,450,2.3
4,610,2.3


In [897]:
# Assign the first row as column headers
case_weights.columns = case_weights.iloc[0]  # Use first row as column names
case_weights = case_weights[1:].reset_index(drop=True)  # Remove the first row

# Display the cleaned headers
print("Updated Column Headers:")
print(case_weights.columns)

# Display first few rows to verify
print(case_weights.head())


Updated Column Headers:
Index(['Item No.', 'Case Weight (lb)'], dtype='object', name=0)
0 Item No. Case Weight (lb)
0      410                2
1      440              2.5
2      450              2.3
3      610              2.3
4      612              2.3


In [898]:
print(case_weights.head())

0 Item No. Case Weight (lb)
0      410                2
1      440              2.5
2      450              2.3
3      610              2.3
4      612              2.3


In [899]:
case_weights.columns = [
    "ProductCode","WeightPerBox"
]

In [900]:
#Handle missing values
print(case_weights.isnull().sum())


ProductCode     0
WeightPerBox    0
dtype: int64


In [901]:
#Convert data types
case_weights["ProductCode"] = case_weights["ProductCode"].astype("category")
case_weights["WeightPerBox"] = case_weights["WeightPerBox"].astype("float32")


In [902]:
print(case_weights.dtypes)

ProductCode     category
WeightPerBox     float32
dtype: object


In [903]:
case_weights = case_weights.drop_duplicates()
case_weights.shape

(395, 2)

In [906]:
# Merge Case Weights into Sales Data to get missing WeightPerCase values
historical_sales_df = historical_sales.merge(case_weights[["ProductCode", "WeightPerBox"]], on="ProductCode", how="left", suffixes=("", "_case_weights"))


In [907]:

# Fill missing WeightPerCase values in Sales with values from Case Weights
historical_sales_df["WeightPerBox"] = historical_sales_df["WeightPerBox"].mask(
    (historical_sales_df["WeightPerBox"] == 0) | (historical_sales_df["WeightPerBox"].isnull()),
    historical_sales_df["WeightPerBox_case_weights"]
)


In [908]:
#I want to drop WeightPerCase_case_weights - becaus eit may not be the most updated info

# Drop the "WeightPerCase_case_weights" column from the sales table if it exists
if "WeightPerBox_case_weights" in historical_sales_df.columns:
    historical_sales_df.drop(columns=["WeightPerBox_case_weights"], inplace=True)

# Verify the column is removed
print("Remaining columns in Fact_Sales:", historical_sales_df.columns)


Remaining columns in Fact_Sales: Index(['Year', 'Month', 'Business', 'Channel', 'CustomerCode', 'State',
       'ProductCode', 'ProductCategory', 'ProductGroup', 'Boxes',
       'WeightPerBox', 'TotalWeight', 'Revenue', 'TransactionType'],
      dtype='object')


In [909]:
historical_sales_df.head

<bound method NDFrame.head of        Year  Month    Business       Channel CustomerCode State  ProductCode  \
0      2020      1  Business A  Food Service       700002    MA        15040   
1      2020      1  Business A  Food Service       700007    MI        15040   
2      2020      1  Business A  Food Service       700052    NY        15040   
3      2020      1  Business A  Food Service       700062    NH        10041   
4      2020      1  Business A  Food Service       700062    NH        15040   
...     ...    ...         ...           ...          ...   ...          ...   
14221  2018     12  Business A  Food Service       900030    IL        50010   
14222  2018     12  Business A  Food Service       900030    IL        50015   
14223  2018     12  Business A  Food Service       900030    IL        50040   
14224  2018     12  Business A  Food Service       900030    IL        60004   
14225  2018     12  Business B  Food Service       900030    IL        97404   

      Pro

In [935]:
#historical_sales_df.head

In [923]:
# Ensure Year and Month exist and are in correct format
if "Year" in historical_sales_df.columns and "Month" in historical_sales_df.columns:
    # Convert Year and Month to integers if not already
    historical_sales_df["Year"] = historical_sales_df["Year"].astype(int)
    historical_sales_df["Month"] = historical_sales_df["Month"].astype(int)

    # Create MonDate column (First day of the month YYYY-MM-01)
    historical_sales_df["MonthDate"] = pd.to_datetime(historical_sales_df[["Year", "Month"]].assign(day=1))

# Verify the new column
print(historical_sales_df[["Year", "Month", "MonthDate"]].head())


# Sort by Year and Month
historical_sales_df = historical_sales_df.sort_values(by=["Year", "Month"]).reset_index(drop=True)

# Verify the structure
print(historical_sales_df.head())


   Year  Month  MonthDate
0  2018      1 2018-01-01
1  2018      1 2018-01-01
2  2018      1 2018-01-01
3  2018      1 2018-01-01
4  2018      1 2018-01-01
   Year  Month    Business       Channel CustomerCode State  ProductCode  \
0  2018      1  Business A  Food Service       700007    MI        15040   
1  2018      1  Business A  Food Service       700052    NY        15040   
2  2018      1  Business A  Food Service       700062    NH        15040   
3  2018      1  Business A  Food Service       700075    CT        15040   
4  2018      1  Business A  Food Service       700077    CT        15040   

  ProductCategory ProductGroup  Boxes  WeightPerBox   TotalWeight  \
0         Branded     Crackers   4800          4.69  22512.000000   
1         Branded     Crackers   3072          4.69  14407.679688   
2         Branded     Crackers   8932          4.69  41891.078125   
3         Branded     Crackers   3456          4.69  16208.639648   
4         Branded     Crackers   3072     

In [924]:
#Calculating Price per Box
# Ensure Quantity (Boxes) and TotalAmt are numeric
historical_sales_df["Boxes"] = pd.to_numeric(historical_sales_df["Boxes"], errors="coerce")
historical_sales_df["Revenue"] = pd.to_numeric(historical_sales_df["Revenue"], errors="coerce")

# Calculate Price Per Box
historical_sales_df["PricePerBox"] = historical_sales_df["Revenue"] / historical_sales_df["Boxes"]

# Verify the calculation
print(historical_sales_df[["Boxes", "Revenue", "PricePerBox"]].head())


   Boxes       Revenue  PricePerBox
0   4800  50928.000000        10.61
1   3072  33576.960938        10.93
2   8932  97626.757812        10.93
3   3456  36668.160156        10.61
4   3072  33576.960938        10.93


In [936]:
#historical_sales_df.head


In [926]:
# Define the new column order
new_column_order = [
    "Year", "Month","MonthDate", "Business", "Channel","CustomerCode","State","ProductCode",
    "ProductCategory", "ProductGroup","TransactionType","Boxes","WeightPerBox","PricePerBox","TotalWeight", "Revenue"]

# Reorder the columns
historical_sales_df = historical_sales_df[new_column_order]

# Verify the new order
print(historical_sales_df.head())

   Year  Month  MonthDate    Business       Channel CustomerCode State  \
0  2018      1 2018-01-01  Business A  Food Service       700007    MI   
1  2018      1 2018-01-01  Business A  Food Service       700052    NY   
2  2018      1 2018-01-01  Business A  Food Service       700062    NH   
3  2018      1 2018-01-01  Business A  Food Service       700075    CT   
4  2018      1 2018-01-01  Business A  Food Service       700077    CT   

   ProductCode ProductCategory ProductGroup TransactionType  Boxes  \
0        15040         Branded     Crackers     Normal Sale   4800   
1        15040         Branded     Crackers     Normal Sale   3072   
2        15040         Branded     Crackers     Normal Sale   8932   
3        15040         Branded     Crackers     Normal Sale   3456   
4        15040         Branded     Crackers     Normal Sale   3072   

   WeightPerBox  PricePerBox   TotalWeight       Revenue  
0          4.69        10.61  22512.000000  50928.000000  
1          4.69 

In [927]:
historical_sales_df.to_csv("Fact_Sales.csv", index=False)


In [929]:
# Dim_Date: Create a Date Dimension Table
Dim_Date = pd.DataFrame({
    "DateKey": pd.date_range(start="2018-01-01", periods=365*5, freq="D").strftime("%Y-%m-%d")
})
Dim_Date["Year"] = pd.to_datetime(Dim_Date["DateKey"]).dt.year
Dim_Date["Month"] = pd.to_datetime(Dim_Date["DateKey"]).dt.month
Dim_Date["Quarter"] = pd.to_datetime(Dim_Date["DateKey"]).dt.quarter
Dim_Date.to_csv("Dim_Date.csv", index=False)


In [928]:
#Let create the dim tables
# Step 1: Create Dimension Tables

# Step 1: Create Dim_Customer with unique Customer Codes
Dim_Customer = historical_sales_df[["CustomerCode"]].drop_duplicates()

# Step 2: Assign the most common State per CustomerCode
Dim_Customer["State"] = historical_sales_df.groupby("CustomerCode")["State"].transform(lambda x: x.mode()[0] if not x.mode().empty else "Unknown")

# Step 3: Save to CSV
Dim_Customer.to_csv("Dim_Customer.csv", index=False)

# Verify the output
print(Dim_Customer.head())


# Dim_Product: Unique Products with Categories & Weights
Dim_Product = historical_sales_df[["ProductCode", "ProductGroup", "ProductCategory", "WeightPerBox", "PricePerBox"]]

# Ensure ProductCode is not null and drop duplicates
Dim_Product = Dim_Product.dropna(subset=["ProductCode"]).drop_duplicates()

# Convert numeric columns
Dim_Product["WeightPerBox"] = pd.to_numeric(Dim_Product["WeightPerBox"], errors="coerce")
Dim_Product["PricePerBox"] = pd.to_numeric(Dim_Product["PricePerBox"], errors="coerce")

# Save to CSV
Dim_Product.to_csv("Dim_Product.csv", index=False)

# Verify output
print(Dim_Product.head())


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_35897/2549266386.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  Dim_Customer["State"] = historical_sales_df.groupby("CustomerCode")["State"].transform(lambda x: x.mode()[0] if not x.mode().empty else "Unknown")


  CustomerCode State
0       700007    MI
1       700052    NY
2       700062    NH
3       700075    CT
4       700077    CT
   ProductCode ProductGroup ProductCategory  WeightPerBox  PricePerBox
0        15040     Crackers         Branded          4.69        10.61
1        15040     Crackers         Branded          4.69        10.93
2        15040     Crackers         Branded          4.69        10.93
3        15040     Crackers         Branded          4.69        10.61
5        15040     Crackers         Branded          4.69        12.54


In [770]:
Fact_Sales = historical_sales_df

In [771]:
# Define the correct chronological order for months
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Convert Month to categorical with correct order
Fact_Sales["Month"] = pd.Categorical(Fact_Sales["Month"], categories=month_order, ordered=True)

# Convert Year to integer if not already
Fact_Sales["Year"] = Fact_Sales["Year"].astype(int)

# Sort by Year first, then by Month (which is now correctly ordered)
Fact_Sales = Fact_Sales.sort_values(by=["Year", "Month"]).reset_index(drop=True)

# Verify the sorting
print(Fact_Sales[["Year", "Month"]].drop_duplicates())


       Year Month
0      2018   Jan
440    2018   Feb
894    2018   Mar
1343   2018   Apr
1822   2018   May
2283   2018   Jun
2660   2018   Jul
3025   2018   Aug
3567   2018   Sep
3980   2018   Oct
4537   2018   Nov
5123   2018   Dec
5490   2019   Jan
5943   2019   Feb
6346   2019   Mar
6772   2019   Apr
7255   2019   May
7668   2019   Jun
8032   2019   Jul
8406   2019   Aug
8930   2019   Sep
9371   2019   Oct
9821   2019   Nov
10301  2019   Dec
10719  2020   Jan
11139  2020   Feb
11493  2020   Mar
11839  2020   Apr
12009  2020   May
12165  2020   Jun
12356  2020   Jul
12645  2020   Aug
12909  2020   Sep
13238  2020   Oct
13600  2020   Nov
13908  2020   Dec


In [937]:
#Fact_Sales.head(29)

In [774]:
Fact_Sales.to_csv("Fact_Sales.csv", index=False)